# Find proteins outside the 8q loss event (trans) that are commonly different between samples with and without the event

# Only looking at interacting proteins

## Setup

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
CHROMOSOME = "8"
ARM = "q"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects_filtered.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,brca_Database_ID,lscc_Database_ID,luad_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
AAR2,NP_001258803.1,NP_001258803.1,NP_001258803.1,NP_001258803,0.094645,0.026602,0.862858,0.987154,0.371156,0.913140,0.258099,0.195941,-0.026980,-0.008821,0.245226,-0.061427
ABL1,NP_009297.2|NP_005148.2,NP_009297.2|NP_005148.2,NaN,NP_005148,0.191034,0.900638,0.338168,0.613746,NaN,0.913140,-0.238002,-0.024027,0.102524,-0.131990,NaN,0.059847
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001003407,0.362542,0.601791,0.956188,0.816619,0.515100,0.913140,-0.332782,-0.075404,0.017450,-0.144921,-0.649705,-0.093721
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001309811,0.362542,0.601791,0.956188,0.816619,0.515100,0.913140,-0.332782,-0.075404,0.017450,-0.144921,-0.649705,-0.091972
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001003407,0.362542,0.601791,0.956188,0.816619,0.745737,0.913140,-0.332782,-0.075404,0.017450,-0.144921,-0.225314,-0.093721
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001309811,0.362542,0.601791,0.956188,0.816619,0.745737,0.913140,-0.332782,-0.075404,0.017450,-0.144921,-0.225314,-0.091972
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001003407,0.362542,0.601791,0.956188,0.816670,0.515100,0.913140,-0.332782,-0.075404,0.017450,0.191611,-0.649705,-0.093721
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_002304.3|NP_001339371.1|NP_001309821.1|NP_0...,NP_001309811,0.362542,0.601791,0.956188,0.816670,0.515100,0.913140,-0.332782,-0.075404,0.017450,0.191611,-0.649705,-0.091972
ABLIM1,NP_001309811.1|NP_001309815.1,NP_001003407.1|NP_002304.3|NP_001309812.1|NP_0...,NP_001309811.1|NP_001309813.1|NP_001339369.1|N...,NP_001003407,0.362542,0.601791,0.956188,0.816670,0.745737,0.913140,-0.332782,-0.075404,0.017450,0.191611,-0.225314,-0.093721


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,AAR2,NP_001258803.1,0.094645,0.258099
1,brca,ABL1,NP_009297.2|NP_005148.2,0.191034,-0.238002
2,brca,ACACA,NP_942133.1|NP_942134.1|NP_942135.1,0.328407,-0.423023
3,brca,ADSL,NP_000017.1|NP_001304852.1|NP_001116850.1,0.392194,0.202071
4,brca,AGO2,NP_036286.2|NP_001158095.1,0.006183,0.402224
5,brca,AGR2,NP_006399.1,0.004639,-1.868970
6,brca,AJUBA,NP_116265.1|NP_001276026.1,0.932205,-0.037392
7,brca,AKAP5,NP_004848.3,0.020140,-1.102004
8,brca,AKT1,NP_005154.2,0.000785,-0.674357
9,brca,AKT1S1,NP_001092102.1|NP_115751.3,0.004639,-0.615393


## Select the proteins with a significant change

In [6]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)

In [7]:
prots.groupby("cancer_type").count()[["protein"]]

,protein
cancer_type,
brca,96
colon,27
hnscc,2
lscc,7
luad,3


## Find how many cancers each protein was different in

In [8]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
PTK2,"[brca, colon, luad]",3
AGO2,"[brca, colon]",2
C21orf59,"[brca, colon]",2
FBL,"[brca, colon]",2
HSPE1,"[brca, colon]",2
MRPL40,"[brca, colon]",2
SMC1A,"[brca, colon]",2
TIMM13,"[brca, colon]",2
ZFYVE19,"[brca, colon]",2


In [9]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

3      1
2     10
1    112
Name: num_cancers, dtype: int64

## Save results

In [10]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}_filtered.tsv"
prots_summary.to_csv(output_file, sep="\t")